# A/B-тестирование нововведений в рекомендательную систему

**Цель исследования** — провести оценку результатов A/B-теста и корректность его проведения в соответствии с техническим заданием.

**Ход исследования**

Данные представлены в виде нескольких таблиц, содержащих список новых пользователей и информацию об их действиях, а также вспомогательную информацию - список участников всех тестов и календарь маркетинговых событий. Необходимо провести обзор и предобработку данных - обработать возможные дубликаты и пропуски, при необходимости поменять тип данных и создать дополнительные столбцы. Затем будет проведена оценка коррекности проведения теста - данные будут сверены с требованиями технического задания и календарём маркетинговых активностей. После будет проведён исследовательский анализ данных, в ходе которого будет изучено распределение данных и конверсия воронки событий, а последним этапом станет анализ результатов проведённого теста. По результатам исследования будет сформирован общий вывод.

Таким образом, исследование будет разделено на следующие этапы:

1. [**Обзор и предобработка данных**](#1)
2. [**Иследовательский анализ данных**](#2)
3. [**Анализ результатов тестирования**](#3)
4. [**Общий вывод**](#4)
___

**Техническое задание**

- Название теста: `recommender_system_test`;
- группы: А — контрольная, B — новая платёжная воронка;
- дата запуска: 2020-12-07;
- дата остановки набора новых пользователей: 2020-12-21;
- дата остановки: 2021-01-04;
- аудитория: 15% новых пользователей из региона EU;
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- ожидаемое количество участников теста: 6000.
- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    - конверсии в просмотр карточек товаров — событие `product_page`,
    - просмотры корзины — `product_cart`,
    - покупки — `purchase`.

<a id='1'></a>
## Обзор и предобработка данных
Сначали импортируем все необходимые таблицы с данными и рассмотрим каждую по отдельности.

In [1]:
# импортируем библиотеки для работы с данными

import os
import pandas as pd
import numpy as np
from scipy import stats as st
import math as mth
import seaborn as sns
import plotly.express as px
from matplotlib import pyplot as plt
from plotly import graph_objects as go

# настройки отображения и рабочей среды

pd.options.display.float_format = '{:,.2f}'.format
pd.options.mode.chained_assignment = None
sns.set_style('darkgrid')
sns.set(font_scale = 2)
plt.rcParams.update({'axes.labelsize': 15,'axes.titlesize': 25})
os.chdir('C:\\Users\\dmitr\\GitHub\\Practicum-Data-Analysis\\datasets')

In [2]:
# загружаем данные

users = pd.read_csv('final_ab_new_users.csv') # новые пользователи
user_events = pd.read_csv('final_ab_events.csv') # действия новых пользователей
test_participants = pd.read_csv('final_ab_participants.csv') # список участников разных AB тестов
marketing_events = pd.read_csv('ab_project_marketing_events.csv') # календарь маркетинговых событий

___
В ходе предобработки и обзора данных будет паралельно проводится

### Новые пользователи

In [3]:
def overview(df):
    
    '''функция для вывода общей информации, 
       описательной статистики и нескольких случайных строк'''
    
    df.info()
    display(
        df.describe(include='all'), 
        df.sample(5, random_state=0)
    )

# таблица с пользователями    
    
overview(users)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


,user_id,first_date,region,device
count,61733,61733,61733,61733
unique,61733,17,4,4
top,D72A72121175D8BE,2020-12-21,EU,Android
freq,1,6290,46270,27520


,user_id,first_date,region,device
9145,8C07103A3DCCA5CB,2020-12-14,EU,Android
33709,9A7EB0465E2DC590,2020-12-10,CIS,Android
7768,1D400F27F375EDDC,2020-12-14,CIS,PC
54387,006E3D14743030D4,2020-12-13,EU,Android
10017,9A79AAC8B0EA52AE,2020-12-14,EU,Mac


___
Таблица содержит 61,733 строку и 4 столбца, явные пропуски отсутствуют.

Описание данных из документации:

`final_ab_new_users.csv` — пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.

Структура таблицы:

- `user_id` — идентификатор пользователя;
- `first_date` — дата регистрации;
- `region` — регион пользователя;
- `device` — устройство, с которого происходила регистрация.
___

Дата регистрации представлены в виде текста, для корректных расчётов необходимо преобразование:

In [4]:
# меняем тип данных

users['first_date'] = pd.to_datetime(users['first_date'])

Из описательной статистики таблицы видно, что все пользователи уникальны. Посмотрим на значения остальных столцов:

In [5]:
# посмотрим на даты регистрации

users['first_date'].value_counts()

2020-12-21    6290
2020-12-14    5654
2020-12-07    5595
2020-12-13    4691
2020-12-20    4288
2020-12-12    3963
2020-12-19    3617
2020-12-18    3365
2020-12-08    3239
2020-12-22    3083
2020-12-10    3076
2020-12-17    3048
2020-12-15    3043
2020-12-11    2390
2020-12-23    2180
2020-12-16    2110
2020-12-09    2101
Name: first_date, dtype: int64

- Абсолютное большинство пользователей зарегестрировались в последний день рассматриваемого периода.

In [6]:
# посмотрим на регионы

users['region'].value_counts()

EU           46270
N.America     9155
CIS           3155
APAC          3153
Name: region, dtype: int64

- В данных присутствуют пользователи из четырёх регионов, большая часть представляет регион `EU`

In [7]:
# посмотрим на пользовательские устройства

users['device'].value_counts()

Android    27520
PC         15599
iPhone     12530
Mac         6084
Name: device, dtype: int64

- Подавляющее больщинство пользователей используют устройство на базе `Android`

### Действия новых пользователей

In [8]:
# таблица с логами пользовательских событий

overview(user_events)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


,user_id,event_dt,event_name,details
count,440317,440317,440317,"62,740.00"
unique,58703,267268,4,NaN
top,A3917F81482141F2,2020-12-23 02:37:24,login,NaN
freq,36,10,189552,NaN
mean,NaN,NaN,NaN,23.88
std,NaN,NaN,NaN,72.18
min,NaN,NaN,NaN,4.99
25%,NaN,NaN,NaN,4.99
50%,NaN,NaN,NaN,4.99
75%,NaN,NaN,NaN,9.99


,user_id,event_dt,event_name,details
390825,355E781F2D95012A,2020-12-22 16:15:15,login,NaN
82196,25CDE8C43F052564,2020-12-15 16:12:06,product_cart,NaN
211107,8857321F4D97B089,2020-12-22 15:56:32,product_page,NaN
182271,6F28139FA1E8EF98,2020-12-18 10:11:55,product_page,NaN
380348,CD104B18BF9D39BE,2020-12-21 21:02:39,login,NaN


___
Таблица содержит 440,317 строк и 4 столбца. Явные пропуски наблюдаются только в столбце `details`.

Описание данных согласно документации:

`final_ab_events.csv` — действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

Структура таблицы:

- `user_id` — идентификатор пользователя;
- `event_dt` — дата и время покупки;
- `event_name` — тип события;
- `details` — дополнительные данные о событии. Например, для покупок, `purchase,` в этом поле хранится стоимость покупки в долларах.
____

- Количество уникальных пользователей таблицы с пользовательскими событиями (*58,703*) не совпадает с количеством из таблицы со всеми новыми пользователями (*61,733*) - это указывает на то, что часть пользователей не совершали никаких активностей, что должно быть учтено при анализе результатов теста.

Даты в этой таблице также представлены в виде текста, сделаем преобразование:

In [9]:
# меняем тип данных

user_events['event_dt'] = pd.to_datetime(user_events['event_dt'])

Изучим уникальные типы событий:

In [10]:
# смотрим уникальные значения

user_events['event_name'].value_counts()

login           189552
product_page    125563
purchase         62740
product_cart     62462
Name: event_name, dtype: int64

- Логи пользовательских событий содержат четыре уникальных значения - `login` (*вход в учётную запись*), `product_page` (*просмотр страницы с товаром*), `product_cart` (*просмотр страницы с корзиной товаров*) и `purchase` (*успешная покупка*). Причём количество событий с покупкой больше, чем количество просмотров корзины, что может сигнализировать об использовании модулей вроде `покупка в один клик`.

На этапе исследовательского анализа данных этот момент будет изучен подробнее с помощью воронки конверсии. 

Дальше попробуем найти закономерность в пропущенных значениях - в документации приведён пример с указанием суммы покупки в столбце `details` для события `purchase`. Проверим, какие ещё могут быть варианты:

In [11]:
# посмотрим на значения `details` (кроме примера из документации)

(
    user_events[user_events['details'].notna()]
    .query('event_name != "purchase"')
)

,user_id,event_dt,event_name,details


Учитывая результаты проверки и числовой формат столбца можем сделать вывод, что столбец `details` используются исключительно для указания суммы покупки. Так как суммы выручки не является одной из целевых метрик для A/B-теста, вместо обработки пропусков мы можем удалить столбец целиком:

In [12]:
user_events.dropna(axis=1, inplace=True)

### Список участников разных A/B-тестов

In [13]:
# список пользователей - участников A/B-тестирования

overview(test_participants)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


,user_id,group,ab_test
count,18268,18268,18268
unique,16666,2,2
top,0FDFDA0B2DEC2D91,A,interface_eu_test
freq,2,9655,11567


,user_id,group,ab_test
4890,9708BA96BC78EE9F,A,recommender_system_test
9825,6D05D44AB00C84D2,B,interface_eu_test
6727,97248D21983C5DBC,B,interface_eu_test
624,92F843D598252571,A,recommender_system_test
15137,CDB54E138640B8EF,A,interface_eu_test


___
Таблица содержит 18,268 строк и 3 столбца. Явные пропуски отсутствуют. 

Описание данных согласно документации:

`final_ab_participants.csv` — таблица участников тестов.

Структура таблицы:

- `user_id` — идентификатор пользователя;
- `ab_test` — название теста;
- `group` — группа пользователя.
___

В таблице можно увидеть данные пользователей двух разных тестирований - `recommender_system_test` и `interface_eu_test`. Целью исследования является анализ результатов только первого теста, но сначала необходимо проверить данные на отсутствие пересечений - количество уникальных идентификаторов пользователей не совпадает с числом строк, что указывает на наличие дубликатов.

In [14]:
# проверим на явные дубликаты

print(f' Задублированных строк - {test_participants.duplicated().sum()}')

# проверим, что нет пользователей, попавших в оба теста

inter = np.intersect1d(
    test_participants.query('ab_test == "recommender_system_test"')['user_id'].unique(),
    test_participants.query('ab_test == "interface_eu_test"')['user_id'].unique()
)
print(f' Уникальных пользователей, попавших в оба теста - {inter.size}')

 Задублированных строк - 0
 Уникальных пользователей, попавших в оба теста - 1602


Явных дубликатов не выявлено, а вот пересечений довольно много - 1,602 уникальных пользователей являлись участниками двух разных тестирований. В таком случае анализировать влияние новвоведений на пользовательское поведение будет неккоректно, так как группы будет невозможно отличить. Необходимо очистить данные от этих пользователей:

In [15]:
# убираем ошибочные данные

test_participants = (
    test_participants[
        ~test_participants['user_id']
        .isin(inter)]
)

# оставляем только данные целевого теста

test_participants = (
    test_participants
    .query('ab_test == "recommender_system_test"')
)

Теперь таким же образом проверим, что одни и те же пользователи не попадают в разные группы тестирования:

In [16]:
# проверим, что нет пользователей, попавших в обе группы

inter = np.intersect1d(
    test_participants.query('group == "A"')['user_id'].unique(),
    test_participants.query('group == "B"')['user_id'].unique()
)
print(f' Уникальных пользователей, попавших в обе группы - {inter.size}')

 Уникальных пользователей, попавших в обе группы - 0


Посмотрим на данные ещё раз:

In [17]:
overview(test_participants)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5099 entries, 0 to 6700
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  5099 non-null   object
 1   group    5099 non-null   object
 2   ab_test  5099 non-null   object
dtypes: object(3)
memory usage: 159.3+ KB


,user_id,group,ab_test
count,5099,5099,5099
unique,5099,2,1
top,D1ABA3E2887B6A73,A,recommender_system_test
freq,1,2903,5099


,user_id,group,ab_test
5339,D2AB21C40985B8BB,A,recommender_system_test
412,F9DC5907335EF6FE,A,recommender_system_test
4848,14995C7DF64B4C31,B,recommender_system_test
1623,F9B6AD71F73676ED,B,recommender_system_test
2843,8728D15493619DA4,B,recommender_system_test


- Пользователи неравномерно распределены по группам - из 5,099 уникальных пользователей 2,903 находятся в группе `A`, что составляет примерно 57%.


- Целевой показатель количества участников в 6,000 формально был достигнут, но из-за пересечений с другим тестом часть данных не подлежит анализу, что может негативно отразиться на выводах из анализа итоговых результатов. 

Согласно техническому задания, целевая аудитория теста - 15% новых пользователей из региона `EU`. Проверим данные таблицы на соответствие этому требованию:

In [43]:
# посмотрим на распределение пользователей по регионам

test_users = list(test_participants["user_id"])

region_distribution = (
    users
    .query('user_id in @test_users')['region']
    .value_counts()
    .to_frame()
    .merge(
        users['region']
        .value_counts()
        .to_frame(),
        left_index=True,
        right_index=True)
    .rename(columns={'region_x':'test',
                     'region_y':'total'})
    .reindex(columns=['total', 'test'])
)

region_distribution['test'] = (
    round(
        region_distribution['test']/
        region_distribution['total']*100, 2)
    .astype(str) + '%'
)

In [45]:
region_distribution

,total,test
EU,46270,10.26%
N.America,9155,2.44%
APAC,3153,2.28%
CIS,3155,1.74%


- Требование технического задания в формировании целовой аудитории теста из 15% пользователей из региона `EU` не соблюдено - участники теста действительно суммарно составляют около 15% пользователей, но лишь чуть больше 10% из них являются пользователями из `EU`, тогда как остальные распределены по другим регионам.

### Календарь маркетинговых событий

In [20]:
# таблица с маркетинговыми активностями

overview(marketing_events)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


,name,regions,start_dt,finish_dt
count,14,14,14,14
unique,14,6,14,14
top,Christmas&New Year Promo,APAC,2020-12-25,2021-01-03
freq,1,4,1,1


,name,regions,start_dt,finish_dt
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
4,4th of July Promo,N.America,2020-07-04,2020-07-11
11,Dragon Boat Festival Giveaway,APAC,2020-06-25,2020-07-01
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19


___
Таблица содержит 14 строк и 4 столбца. Явные пропуски отсутствуют.

Описание данных согласно документации:

`ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год.

Структура таблицы:

- `name` — название маркетингового события;
- `regions` — регионы, в которых будет проводиться рекламная кампания;
- `start_dt` — дата начала кампании;
- `finish_dt` — дата завершения кампании.
___

Преобразуем данные и проверим, имеются ли в наличии компании, пересекающиеся с регистрацией новых пользователей:

In [21]:
# преобразовываем даты

marketing_events['start_dt'] = (
    pd.to_datetime(marketing_events['finish_dt'])
)
marketing_events['finish_dt'] = (
    pd.to_datetime(marketing_events['finish_dt'])
)

# посмотрим компании декабря и января

interscecting_marketing = marketing_events[
    (marketing_events['start_dt'].dt.month.isin([1, 12]))|
    (marketing_events['finish_dt'].dt.month.isin([1, 12]))
]

interscecting_marketing

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2021-01-03,2021-01-03
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-12-01,2020-12-01
10,CIS New Year Gift Lottery,CIS,2021-01-07,2021-01-07


- Маркетинговая компании `Christmas&New Year Promo` пересекается с датами проведения A/B-тестирования, что может повлиять на пользовательское поведение и исказить вывода анализа результатов теста.

Остальные компании либо не совпадают по датам, либо не имеют отношение к региону `EU`.

### Выводы

**Техническое задание**

- Название теста: `recommender_system_test`; 
- группы: А — контрольная, B — новая платёжная воронка;
- дата запуска: 2020-12-07;
- дата остановки набора новых пользователей: 2020-12-21;
- дата остановки: 2021-01-04;
- аудитория: 15% новых пользователей из региона EU;
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- ожидаемое количество участников теста: 6000.
- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    - конверсии в просмотр карточек товаров — событие `product_page`,
    - просмотры корзины — `product_cart`,
    - покупки — `purchase`.

<a id='2'></a>
## Исследовательский анализ данных
- Проведите исследовательский анализ данных:
    - Количество событий на пользователя одинаково распределены в выборках?
    - Как число событий в выборках распределено по дням?
    - Как меняется конверсия в воронке в выборках на разных этапах?
    - Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?

<a id='3'></a>
## Оценка результатов A/B-тестирования

- Оцените результаты A/B-тестирования
    - Что можно сказать про результаты A/В-тестирования?
    - Проверьте статистическую разницу долей z-критерием.
- Опишите выводы по этапу исследовательского анализа данных и по проведённой оценке результатов A/B-тестирования. Сделайте общее заключение о корректности проведения теста.

<a id='5'></a>
##  Общий вывод